## Tutorial for Statistical Analysis on Simulations of PTP1B. 

In this jupyter notebook we will use the stat_modelling.py module to identify differences in the molecular interactions across PTP1B
when the WPD-loop of PTP1B is in the Closed state, versus when the WPD-loop is in the Open state.
This notebook will also cover all the pre- and post-processing steps requireds to prepare, analyse and visualise the results.

The dataset used here is for WT PTP1B and is the same data as what was used in the manuscript. 

<center><img src="miscellaneous/PTP1B_Stat_Analysis_Banner.png" alt="Drawing" style="width: 70%" /></center>

In [1]:
import sys # note temporary... 
sys.path.append("..") # note temporary...

from key_interactions_finder import pycontact_processing
from key_interactions_finder import data_preperation
from key_interactions_finder import stat_modelling
from key_interactions_finder import post_proccessing
from key_interactions_finder import pymol_projections

### Step 1. Process PyContact files with the pycontact_processing.py module 

In this section we will work with the PyContact output files generated. 
Here we will merge our seperate runs together and remove any false interactions that can be generated by the PyContact library. 

In [ ]:
pycontact_files_horizontal = ["PyContact_Per_Frame_Interactions_Block1.csv", "PyContact_Per_Frame_Interactions_Block2.csv",
                              "PyContact_Per_Frame_Interactions_Block3.csv", "PyContact_Per_Frame_Interactions_Block4.csv",
                              "PyContact_Per_Frame_Interactions_Block5.csv", "PyContact_Per_Frame_Interactions_Block6.csv",
                              "PyContact_Per_Frame_Interactions_Block7.csv", "PyContact_Per_Frame_Interactions_Block8.csv",
                              "PyContact_Per_Frame_Interactions_Block9.csv", "PyContact_Per_Frame_Interactions_Block10.csv",
                              "PyContact_Per_Frame_Interactions_Block11.csv", "PyContact_Per_Frame_Interactions_Block12.csv",
                              "PyContact_Per_Frame_Interactions_Block13.csv", "PyContact_Per_Frame_Interactions_Block14.csv",
                              "PyContact_Per_Frame_Interactions_Block15.csv", "PyContact_Per_Frame_Interactions_Block16.csv",
                              "PyContact_Per_Frame_Interactions_Block17.csv", "PyContact_Per_Frame_Interactions_Block18.csv",
                              "PyContact_Per_Frame_Interactions_Block19.csv", "PyContact_Per_Frame_Interactions_Block20.csv",
                              ]

pycontact_dataset = pycontact_processing.PyContactInitializer(
    pycontact_files=pycontact_files_horizontal,
    multiple_files=True,
    merge_files_method="horizontal",  
    remove_false_interactions=True,
    in_dir="datasets/PTP1B_Data/example_horizontal_data/",
)

In [ ]:
# As outputted above, we can inspect the newly prepared dataset by accessing the '.prepared_df' class attribute as follows:
pycontact_dataset.prepared_df

### Step 2 Prepare the Dataset for Statistical Analysis with the data_preperation.py module. 

In this step, we take our dataframe and merge our per frame classifications file to it.

We can also optionally perform several forms of filtering on the PyContact features to select what types of interactions we would like to study.  

In [ ]:
# First we generate an instance of the SupervisedFeatureData class (because we have per frame target labels).
classifications_file = "datasets/PTP1B_Data/example_horizontal_data/WT_PTP1B_Class_Assingments.txt"

supervised_dataset = data_preperation.SupervisedFeatureData(
    input_df=pycontact_dataset.prepared_df,
    target_file=classifications_file,
    is_classification=True,
    header_present=False # If your target_file has a header present, set to True.
)

In [ ]:
# As stated above to access the newly generated dataframe we can use the class attribute as follows
supervised_dataset.df_processed

##### Optional Feature Filtering

In the above dataframe we have 2791 columns (so 2790 features + 1 target). We can take all of these forward for the stastical analysis or we can perform some filtering in advance (the choice is yours). 
There are five built in filtering methods available to you to perform filtering:

1. **filter_by_occupancy(min_occupancy)** - Remove features that have an %occupancy less than the provided cut-off. %Occupancy is the % of frames with a non 0 value, i.e. the interaction is present in that frame.

2. **filter_by_interaction_type(interaction_types_included)** - PyContact defines four types of interactions ("Hbond", "Saltbr", "Hydrophobic", "Other"). You select the interactions your want to include.

3. **filter_by_main_or_side_chain(main_side_chain_types_included)** - PyContact can also define if each interaction is primarily from the backbone or side-chain for each residue. You select the interaction combinations you want to include. Options are: "bb-bb", "sc-sc", "bb-sc", "sc-bb". Where bb = backbone and sc = sidechain.

4. **filter_by_avg_strength(average_strength_cut_off)** - PyContact calculates a per frame contact score/strength for each interaction. You can filter features by the average score. Values below the cut-off are removed. 

5. **filter_by_occupancy_by_class(min_occupancy)** - Special alternative to the the standard filter features by occupancy method. %occupancy is determined for each class (as opposed to whole dataset), meaning only observations from 1 class have to meet the cut-off to keep the feature. Only avaible to datasets with a categorical target variable (classification). 


Finally if at any point in time you want to reset any filtering you've already performed, you can use the following method: 

6. **reset_filtering()** 


In [ ]:
# An example of filtering the dataset using the 4 available methods. 
supervised_dataset.reset_filtering() 
print(f"Number of features before any filtering: {len(supervised_dataset.df_processed.columns)}")

# Features with a %occupancy of less than 25% are removed. 
supervised_dataset.filter_by_occupancy_by_class(min_occupancy=25)
print(f"Number of features after filtering by occupancy: {len(supervised_dataset.df_filtered.columns)}")

# Remove features with interaction type "Other".  
supervised_dataset.filter_by_interaction_type(
    interaction_types_included=["Hbond", "Saltbr", "Hydrophobic"]) 
print(f"Number of features after filtering by interaction type: {len(supervised_dataset.df_filtered.columns)}")

# No filtering performed here as all possible combinations are included. 
supervised_dataset.filter_by_main_or_side_chain(
    main_side_chain_types_included=["bb-bb", "sc-sc", "bb-sc", "sc-bb"]  
)
print(f"Number of features after NOT filtering by main or side chain: {len(supervised_dataset.df_filtered.columns)}")

# Features with an average interaction strength less than 0.5 will be removed. 
supervised_dataset.filter_by_avg_strength(
    average_strength_cut_off=0.5,  
)
print(f"Number of features after filtering by average interaction scores: {len(supervised_dataset.df_filtered.columns)}")

Now if we look at the class attributes of our SupervisedFeatureData() instance (we called it: supervised_dataset) using the special "\_\_dict__" method we can see two dataframes we could use in the stastical analysis to follow. 

In [ ]:
supervised_dataset.__dict__.keys()

They are: 
- 'df_processed' - The unfiltered dataframe, 2791 features
- 'df_filtered' - The filtered dataframe. Less than 2791 features. 

In the following section we will use the filtered dataframe but either dataframe could be justified based on your goals. 

### Step 3 Perform the Statistical Analysis with the stat_modelling.py module. 

Now we will perform the actual statistical modelling to compare the differences for each feature when the WPD-loop is in the closed and open state. 

This module only works with two classes (i.e. binary classification). You can select what classes you want to use in the next code block. 

With this module, we can calculate two different metrics to evaluate how different/similar each feature is when the protein is in the closed WPD-loop conformation or the open-WPD-loop conformation: 

1. The Jensen-Shannon distance. This xxxx. After first generating probability distributions of the interaction strengths for each feature when in the two states provided, this is calculated using the [implementation available in SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html).
2. Mutual information: This calculates the xxxx. This is calculated using the [implementation available in Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html).

In both cases, the higher the score, the more "different" the feature is when in the two different states.  

In [ ]:
stat_model = stat_modelling.ProteinStatModel(
    dataset=supervised_dataset.df_filtered, 
    class_names=["Closed", "Open"], # select the two class labels to compare. Has to be 2 labels. 
    out_dir="outputs/PTP1B_stat_analysis",
    interaction_types_included=["Hbond", "Saltbr", "Hydrophobic", "Other"] 
)

In [ ]:
# First we can calculate the Jensen-Shannon distances (Took me about 20 seconds).
stat_model.calc_js_distances()

In [ ]:
# Now we can calculate the mutual informations (Took me about a minute).
stat_model.calc_mutual_info_to_target()
# TODO - IF I AM SAVING DATA HERE, I NEED IT TO PRINT THAT OUT. 

In [ ]:
# As printed above we can access the results from these calculations from the class instance's (we called it stat_model) attributes. 
mi_results = stat_model.mutual_infos
js_results = stat_model.js_distances
# stat_model.__dict__.keys() # uncomment to see all attributes available. 

### Step 4 Work up the Statistical Analysis with the post_proccessing.py module. 

In this module we have access to 3 methods to analyse the results in more detail.

1. We can use generate these results to be at the per-residue level, by summing (and then normalising) the feature scores that each residue is involved in.
This can allow us to identify residues which seem to have the largest overall difference in interactions between each state. 

2. We can also try to predict the "direction" each feature favours. 

3. We can obtain the probability distrubitions of the interaction scores for a selected set of features to visually compare the distributions. 

In [ ]:
# First generate an instance of the class. 
post_proc = post_proccessing.StatisticalPostProcessor(
    stat_model=stat_model,
    out_dir="outputs/PTP1B_stat_analysis"
)

# As you have already seen in the prior steps, we can take a look at class attributes as follows.
# Note that some of these attributes will be empty until we run the next few code blocks.
post_proc.__dict__.keys()

In [ ]:
# Now we can run the get_per_res_importance() method, changing the stat_method accordingly.
js_per_res_importances = post_proc.get_per_res_importance(
    stat_method="jensen_shannon")
mi_per_res_importances = post_proc.get_per_res_importance(
    stat_method="mutual_information")

In [ ]:
post_proc.estimate_feature_directions()

In [ ]:
# Now we can plot the probability distrubitions for the most different Features. 
x_values, selected_prob_distribs = post_proc.get_probability_distributions(
    number_features=10)

# TODO - DESCRIBE THE OUTPUT, MAKE A NICE PLOTLY GRAPH :) 

### Part 5 Projecting the Results onto Protein Structures with the pymol_projections.py module. 
 
Naturally, we may want to visualise some of the results we have generated above onto a protein structure. We can take advantage of
the functions provided in the pymol_projections.py module to do this. 

As the name suggests this will output [PyMOL](https://pymol.org/) compatible python scripts which can be run to represent the results
at the: 

1. Per feature level. (Cylinders are drawn between each feature, with the cylinder radii marking how strong the relative difference is. 
2. Per residue level. The Carbon alpha of each residue will be depicted as a sphere, with the sphere radii depicting how strong the the relative difference is. 

In [ ]:
# Write PyMOL compatable scripts for the per feature results.
# Simply swap between the two statistical methods as shown below. 
pymol_projections.project_pymol_top_features(
    per_feature_scores=stat_model.js_distances,
    model_name="jenson_shannon",
    numb_features=150,
    out_dir="outputs/PTP1B_stat_analysis"
)

pymol_projections.project_pymol_top_features(
    per_feature_scores=stat_model.mutual_infos,
    model_name="mutual_information",
    numb_features=150,
    out_dir="outputs/PTP1B_stat_analysis"
)

In [ ]:
# Write PyMOL compatable scripts for the per residue results.
# Simply swap between the two statistical methods as shown below. 
pymol_projections.project_pymol_per_res_scores(
    per_res_scores=js_per_res_importances,
    model_name="jenson_shannon",
    out_dir="outputs/PTP1B_stat_analysis"
)

pymol_projections.project_pymol_per_res_scores(
    per_res_scores=mi_per_res_importances,
    model_name="mutual_information",
    out_dir="outputs/PTP1B_stat_analysis"
)

In [ ]:

# TODO ADD Picture of the outputs here as an example. 


### TODO Add below section as example way to calculate distance to target and make nice plot 

In [ ]:
from MDAnalysis.analysis import distances
import MDAnalysis as mda
import numpy as np

In [ ]:
pdb_file = "datasets/PTP1B_data/WT_PTP1B_Phospho_Enzyme_Closed.pdb"
universe = mda.Universe(pdb_file)

catres = "resid 180 and name CA"
all_residues = "resid 0-298 and name CA"

group1 = universe.select_atoms(catres)
group2 = universe.select_atoms(all_residues)

In [ ]:
group2

In [ ]:
dist_arr = distances.distance_array(group1.positions, group2.positions, box=universe.dimensions)

out_file = r"C:\Users\Rory Crean\Dropbox (lkgroup)\Backup_HardDrive\Postdoc\MachLearnConformationalFeatures\Workup\PTP1B_Stats\2_Non_PyMOL_Stats\Distance_toCalpha_D181_Closed.txt"
np.savetxt(out_file, dist_arr.T)